In [7]:
from datetime import datetime
st = datetime.now()

from selenium import webdriver
from PIL import Image
import time
import io
import hashlib
import requests

DRIVER_PATH = 'geckodriver.exe' # FIREFOX: geckodriver || CHROME: chromedriver
SEARCH_TERM = 'FAMILIA'
NUMBER_IMAGES = 100
TARGET_PATH = r'D:\\Imagens\\PROJETO FINAL TESTE\\'

In [8]:
def search_and_download(search_term:str,driver_path:str,target_path:str,number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Firefox(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [9]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [10]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [11]:

search_and_download(
    search_term=SEARCH_TERM,
    driver_path=DRIVER_PATH,
    target_path=TARGET_PATH,
    number_images=100
)

Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
SUCCESS - saved https://coracaofiel.com.br/site/wp-content/uploads/2017/05/no-dia-da-familia-comemorado-nesta-terca-08-especialista-da-dicas-de-como-ter-uma-convivencia-familiar-harmoniosa-e-feliz-_620_.jpg - as D:\\Imagens\\PROJETO FINAL TESTE\\familia\fc387381b9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQnc7WFFn9XnnJ4EzsBLOVkFu0af91qamuF31Hp-XOoq-R46GqF&usqp=CAU - as D:\\Imagens\\PROJETO FINAL TESTE\\familia\927a8a8f69.jpg
SUCCESS - saved https://blog.wpensar.com.br/wp-content/uploads/2017/04/capa-relacao-entre-escola-e-familia.png - as D:\\Imagens\\PROJETO FINAL TESTE\\familia\28b3d374d2.jpg
SUCCESS - saved https://d168rbuicf8uyi.cloudfront.net/wp-content/uploads/2010/07/19174329/amizade-familia-do-coracao-1024x649.jpg - as D:\\Imagens\\PROJETO FINAL TESTE\\familia\438a024ddd.jpg
SUCCESS - saved https://st2.depositphotos.com/1967477/6350/v/450/depositphotos_63509663

In [12]:
en = datetime.now()
print('TEMPO TOTAL: ', (en-st))

TEMPO TOTAL:  0:02:55.567971
